In [ ]:
!pip install gurobipy>=10

import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# I. Basic Scenario

In [ ]:
file_path = '/content/budget problem data.xlsx'
data = pd.read_excel(file_path)

budget = data.iloc[0, 2:5].values
labor_availability = data.iloc[0, 5:].values

project_data = data.iloc[1:].reset_index(drop=True)
npv = project_data['NPV'].values
cost_year_1 = project_data['Cost Year 1'].values
cost_year_2 = project_data['Cost Year 2'].values
cost_year_3 = project_data['Cost Year 3'].values
labor_year_1 = project_data['Labor Year 1'].values
labor_year_2 = project_data['Labor Year 2'].values
labor_year_3 = project_data['Labor Year 3'].values

num_projects = len(project_data)

m = gp.Model("capital_budget")

x = m.addVars(num_projects, vtype=GRB.BINARY, name="x")

m.setObjective(gp.quicksum(npv[i] * x[i] for i in range(num_projects)), sense=GRB.MAXIMIZE)

m.addConstr(gp.quicksum(cost_year_1[i] * x[i] for i in range(num_projects)) <= budget[0], "budget_year_1")
m.addConstr(gp.quicksum(cost_year_2[i] * x[i] for i in range(num_projects)) <= budget[1], "budget_year_2")
m.addConstr(gp.quicksum(cost_year_3[i] * x[i] for i in range(num_projects)) <= budget[2], "budget_year_3")

m.addConstr(gp.quicksum(labor_year_1[i] * x[i] for i in range(num_projects)) <= labor_availability[0], "labor_year_1")
m.addConstr(gp.quicksum(labor_year_2[i] * x[i] for i in range(num_projects)) <= labor_availability[1], "labor_year_2")
m.addConstr(gp.quicksum(labor_year_3[i] * x[i] for i in range(num_projects)) <= labor_availability[2], "labor_year_3")

m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 6 rows, 20 columns and 120 nonzeros
Model fingerprint: 0x70eb4e5f
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [3e+00, 4e+02]
  Objective range  [5e+02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+02, 3e+03]
Found heuristic solution: objective 8403.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 20 columns, 100 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 8860.0000000

Root relaxation: objective 9.560849e+03, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [ ]:
selected_projects = []
for i, v in enumerate(m.getVars()):
    print(v.VarName, v.X)
    if v.X > 0:
        selected_projects.append(i + 1)

print("Selected projects:", selected_projects)

x[0] 0.0
x[1] 1.0
x[2] 1.0
x[3] 0.0
x[4] -0.0
x[5] 1.0
x[6] 1.0
x[7] 1.0
x[8] 1.0
x[9] 1.0
x[10] -0.0
x[11] -0.0
x[12] 0.0
x[13] 1.0
x[14] 1.0
x[15] 1.0
x[16] 0.0
x[17] -0.0
x[18] 1.0
x[19] 1.0
Selected projects: [2, 3, 6, 7, 8, 9, 10, 14, 15, 16, 19, 20]


In [ ]:
print(m.ObjVal)

9293.0


# II: Suppose if we select project 3, we must also select project 4.  

In [ ]:
file_path = '/content/budget problem data.xlsx'
data = pd.read_excel(file_path)

budget = data.iloc[0, 2:5].values
labor_availability = data.iloc[0, 5:].values

project_data = data.iloc[1:].reset_index(drop=True)
npv = project_data['NPV'].values
cost_year_1 = project_data['Cost Year 1'].values
cost_year_2 = project_data['Cost Year 2'].values
cost_year_3 = project_data['Cost Year 3'].values
labor_year_1 = project_data['Labor Year 1'].values
labor_year_2 = project_data['Labor Year 2'].values
labor_year_3 = project_data['Labor Year 3'].values

num_projects = len(project_data)

m = gp.Model("capital_budget")

x = m.addVars(num_projects, vtype=GRB.BINARY, name="x")

m.setObjective(gp.quicksum(npv[i] * x[i] for i in range(num_projects)), GRB.MAXIMIZE)

m.addConstr(gp.quicksum(cost_year_1[i] * x[i] for i in range(num_projects)) <= budget[0], "budget_year_1")
m.addConstr(gp.quicksum(cost_year_2[i] * x[i] for i in range(num_projects)) <= budget[1], "budget_year_2")
m.addConstr(gp.quicksum(cost_year_3[i] * x[i] for i in range(num_projects)) <= budget[2], "budget_year_3")

m.addConstr(gp.quicksum(labor_year_1[i] * x[i] for i in range(num_projects)) <= labor_availability[0], "labor_year_1")
m.addConstr(gp.quicksum(labor_year_2[i] * x[i] for i in range(num_projects)) <= labor_availability[1], "labor_year_2")
m.addConstr(gp.quicksum(labor_year_3[i] * x[i] for i in range(num_projects)) <= labor_availability[2], "labor_year_3")

m.addConstr(x[2] <= x[3], "project_3_implies_project_4")

m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 20 columns and 122 nonzeros
Model fingerprint: 0xb81f2635
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e+02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e+02, 3e+03]
Found heuristic solution: objective 7667.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 20 columns, 102 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 8139.0000000

Root relaxation: objective 9.421146e+03, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent   

In [ ]:
selected_projects = []
for i, v in enumerate(m.getVars()):
    print(v.VarName, v.X)
    if v.X > 0:
        selected_projects.append(i + 1)

print("Selected projects:", selected_projects)

x[0] 1.0
x[1] 1.0
x[2] 1.0
x[3] 1.0
x[4] 0.0
x[5] 1.0
x[6] 1.0
x[7] 1.0
x[8] 1.0
x[9] 0.0
x[10] 1.0
x[11] 0.0
x[12] 0.0
x[13] 0.0
x[14] 0.0
x[15] 1.0
x[16] 1.0
x[17] 0.0
x[18] 1.0
x[19] 0.0
Selected projects: [1, 2, 3, 4, 6, 7, 8, 9, 11, 16, 17, 19]


In [ ]:
print(m.ObjVal)

9178.0


# III: suppose we can do only 4 projects from among projects 1 through 10.

In [ ]:
file_path = '/content/budget problem data.xlsx'
data = pd.read_excel(file_path)

budget = data.iloc[0, 2:5].values
labor_availability = data.iloc[0, 5:].values

project_data = data.iloc[1:].reset_index(drop=True)
npv = project_data['NPV'].values
cost_year_1 = project_data['Cost Year 1'].values
cost_year_2 = project_data['Cost Year 2'].values
cost_year_3 = project_data['Cost Year 3'].values
labor_year_1 = project_data['Labor Year 1'].values
labor_year_2 = project_data['Labor Year 2'].values
labor_year_3 = project_data['Labor Year 3'].values

num_projects = len(project_data)

m = gp.Model("capital_budget")

x = m.addVars(num_projects, vtype=GRB.BINARY, name="x")

m.setObjective(gp.quicksum(npv[i] * x[i] for i in range(num_projects)), GRB.MAXIMIZE)

m.addConstr(gp.quicksum(cost_year_1[i] * x[i] for i in range(num_projects)) <= budget[0], "budget_year_1")
m.addConstr(gp.quicksum(cost_year_2[i] * x[i] for i in range(num_projects)) <= budget[1], "budget_year_2")
m.addConstr(gp.quicksum(cost_year_3[i] * x[i] for i in range(num_projects)) <= budget[2], "budget_year_3")

m.addConstr(gp.quicksum(labor_year_1[i] * x[i] for i in range(num_projects)) <= labor_availability[0], "labor_year_1")
m.addConstr(gp.quicksum(labor_year_2[i] * x[i] for i in range(num_projects)) <= labor_availability[1], "labor_year_2")
m.addConstr(gp.quicksum(labor_year_3[i] * x[i] for i in range(num_projects)) <= labor_availability[2], "labor_year_3")

m.addConstr(gp.quicksum(x[i] for i in range(10)) <= 4, "max_4_projects_from_1_to_10")

m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 20 columns and 130 nonzeros
Model fingerprint: 0x1abbafee
Variable types: 0 continuous, 20 integer (20 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [5e+02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 3e+03]
Found heuristic solution: objective 8198.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 20 columns, 110 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)
Found heuristic solution: objective 8518.0000000

Root relaxation: objective 9.226900e+03, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    

In [ ]:
selected_projects = []
for i, v in enumerate(m.getVars()):
    print(v.VarName, v.X)
    if v.X > 0:
        selected_projects.append(i + 1)

print("Selected projects:", selected_projects)

x[0] -0.0
x[1] -0.0
x[2] 1.0
x[3] -0.0
x[4] -0.0
x[5] -0.0
x[6] -0.0
x[7] 1.0
x[8] 1.0
x[9] 1.0
x[10] 1.0
x[11] -0.0
x[12] -0.0
x[13] 1.0
x[14] 1.0
x[15] 1.0
x[16] 1.0
x[17] 1.0
x[18] 1.0
x[19] 1.0
Selected projects: [3, 8, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20]


In [ ]:
print(m.ObjVal)

9072.0
